In [78]:
# Importing Libraries
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date
from datasets import load_dataset

# Loading Data
dataset = load_dataset("lukebarousse/data_jobs")
df = dataset["train"].to_pandas()

# Data Cleanup
df["job_posted_date"] = pd.to_datetime(df["job_posted_date"])

In [6]:
df[df['salary_year_avg'].notna()]['salary_year_avg']

28        109500.0
77        140000.0
92        120000.0
100       228222.0
109        89000.0
            ...   
785624    139216.0
785641    150000.0
785648    221875.0
785682    157500.0
785692    157500.0
Name: salary_year_avg, Length: 22003, dtype: float64

In [9]:
help(df.apply)

Help on method apply in module pandas.core.frame:

apply(
    func: 'AggFuncType',
    axis: 'Axis' = 0,
    raw: 'bool' = False,
    result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None,
    args=(),
    by_row: "Literal[False, 'compat']" = 'compat',
    engine: "Literal['python', 'numba']" = 'python',
    engine_kwargs: 'dict[str, bool] | None' = None,
    **kwargs
) method of pandas.core.frame.DataFrame instance
    Apply a function along an axis of the DataFrame.

    Objects passed to the function are Series objects whose index is
    either the DataFrame's index (``axis=0``) or the DataFrame's columns
    (``axis=1``). By default (``result_type=None``), the final return type
    is inferred from the return type of the applied function. Otherwise,
    it depends on the `result_type` argument.

    Parameters
    ----------
    func : function
        Function to apply to each column or row.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis along wh

In [12]:
df_salary = df[df['salary_year_avg'].notna()].copy()

def projected_salary(salary):
    return salary * 1.03

df_salary['salary_year_inflated'] = df_salary['salary_year_avg'].apply(projected_salary)

df_salary[['salary_year_avg', 'salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


In [14]:
df_salary['salary_year_inflated'] = df_salary['salary_year_avg'].apply(lambda salary: salary*1.03)

df_salary[['salary_year_avg', 'salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


## ast.literal_eval

- To convert string type column which contains a list, to a list.

In [40]:
df['job_skills'][1]

"['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']"

In [45]:
import ast

type(ast.literal_eval(df['job_skills'][1]))

list

In [89]:
def clean_list(skill_list):
    if pd.notna(skill_list):
        return ast.literal_eval(skill_list)

df['job_skills'] = df['job_skills'].apply(clean_list)

df['job_skills'][1]

['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']

In [ ]:
df['job_skills'] = df['job_skills'].apply(lambda skill_list: ast.literal_eval(skill_list) if pd.notna(skill_list) else skill_list)

['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']

## Calculate projected salary next year

- Senior roles assume 5%
- Other roles assume 3%

In [ ]:
df_salary['salary_year_inflated'] = df_salary['salary_year_avg'].apply(lambda salary: salary*1.03)

df_salary[['salary_year_avg', 'salary_year_inflated']]

In [ ]:
def projected_salary(row):
    if "Senior" in row['job_title_short']:
        return row['salary_year_avg']*1.05
    else:
        return row['salary_year_avg']*1.03


df_salary['salary_year_inflated'] = df_salary.apply(projected_salary, axis= 1)

df_salary[['job_title_short', 'salary_year_avg', 'salary_year_inflated']]

,job_title_short,salary_year_avg,salary_year_inflated
28,Data Scientist,109500.0,112785.00
77,Data Engineer,140000.0,144200.00
92,Data Engineer,120000.0,123600.00
100,Data Scientist,228222.0,235068.66
109,Data Analyst,89000.0,91670.00
...,...,...,...
785624,Data Engineer,139216.0,143392.48
785641,Data Engineer,150000.0,154500.00
785648,Data Scientist,221875.0,228531.25
785682,Data Scientist,157500.0,162225.00


In [125]:
df_salary[df_salary['job_title_short'].isin(['Data Engineer'])]

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills,salary_year_inflated
77,Data Engineer,Data Engineer,"Arlington, VA",via LinkedIn,Full-time,False,Sudan,2023-06-26 14:22:54,False,False,Sudan,year,140000.0,NaN,Intelletec,"['mongodb', 'mongodb', 'python', 'r', 'sql', '...","{'analyst_tools': ['tableau'], 'cloud': ['orac...",144200.00
92,Data Engineer,Remote - Data Engineer - Permanent - W2,Anywhere,via LinkedIn,Full-time,True,"Illinois, United States",2023-02-21 13:29:59,False,True,United States,year,120000.0,NaN,Apex Systems,"['sql', 'python']","{'programming': ['sql', 'python']}",123600.00
146,Data Engineer,Data Engineer Graduate (Real Time Communicatio...,"Seattle, WA",via LinkedIn,Full-time,False,"California, United States",2023-10-21 13:06:16,False,True,United States,year,129500.0,NaN,ByteDance,"['golang', 'scala', 'python', 'java', 'hadoop'...","{'libraries': ['hadoop', 'spark'], 'programmin...",133385.00
307,Data Engineer,Data Engineer,"Austin, TX",via LinkedIn,Full-time,False,"New York, United States",2023-10-03 13:05:31,False,False,United States,year,185000.0,NaN,hackajob,"['nosql', 'sql']","{'programming': ['nosql', 'sql']}",190550.00
339,Data Engineer,Global Market Leading Energy Firm - Staff Data...,United States,via LinkedIn,Full-time,False,"California, United States",2023-09-22 13:07:57,False,False,United States,year,195000.0,NaN,Xcede,None,None,200850.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784102,Data Engineer,Data Engineer III- Data Engineering,"Melbourne, FL",via Ladders,Full-time,False,Georgia,2023-01-09 06:51:17,False,False,United States,year,90000.0,NaN,Health First,"['sql', 'r', 'python', 'c#', 'java', 'db2', 'o...","{'cloud': ['oracle', 'azure'], 'databases': ['...",92700.00
784191,Data Engineer,Data Engineer (Hybrid),United States,via Ai-Jobs.net,Full-time,False,"Florida, United States",2023-04-21 06:06:25,False,True,United States,year,217500.0,NaN,Captivation Software,"['python', 'java', 'elasticsearch', 'aws', 'ka...","{'analyst_tools': ['excel'], 'cloud': ['aws'],...",224025.00
785395,Data Engineer,Senior Consultant - Tech Consulting -Azure Dat...,India,via Ai-Jobs.net,Full-time,False,India,2023-12-19 06:10:41,False,False,India,year,64800.0,NaN,EY,"['sql', 'python', 'azure', 'databricks', 'spar...","{'analyst_tools': ['power bi', 'tableau', 'qli...",66744.00
785624,Data Engineer,Data Analytics Engineer (Hybrid),"Mt Prospect, IL",via Ai-Jobs.net,Full-time,False,"Illinois, United States",2023-08-31 06:02:16,False,True,United States,year,139216.0,NaN,Bosch Group,"['go', 'python', 'r', 'sql', 'oracle', 'window...","{'analyst_tools': ['alteryx', 'power bi', 'tab...",143392.48


In [124]:
df_salary['salary_year_inflated'] = df_salary.apply(lambda row: row['salary_year_avg']*1.05 if 'Senior' in row['job_title_short'] else row['salary_year_avg']*1.03, axis= 1)

df_salary[df_salary['job_title_short'] == 'Senior Data Engineer'][['job_title_short', 'salary_year_avg', 'salary_year_inflated']]

,job_title_short,salary_year_avg,salary_year_inflated
495,Senior Data Engineer,168500.0,176925.0
573,Senior Data Engineer,160000.0,168000.0
657,Senior Data Engineer,165000.0,173250.0
726,Senior Data Engineer,173500.0,182175.0
733,Senior Data Engineer,160000.0,168000.0
...,...,...,...
782562,Senior Data Engineer,182500.0,191625.0
783761,Senior Data Engineer,130000.0,136500.0
784364,Senior Data Engineer,147500.0,154875.0
784947,Senior Data Engineer,150000.0,157500.0
